# Plots of stability regions

In [ ]:
import numpy as np
import scipy.linalg as linalg
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import matplotlib as mpl

from nodepy import rk

from OrderCondition import *

import plot_fkt
plot_fkt.setup_plt()

mpl.rcParams['hatch.linewidth'] = 2.0

In [ ]:
def plot_stab(stab_function,color,fill=None,xmin=-10,xmax=10,yminmax=10,hatch=None,linewith=3):
    p,q = stab_function

    # Convert coefficients to floats for speed
    if p.coeffs.dtype=='object':
        p = np.poly1d([float(c) for c in p.coeffs])
    if q.coeffs.dtype=='object':
        q = np.poly1d([float(c) for c in q.coeffs])
        
    u = np.linspace(xmin,xmax,100)
    v = np.linspace(-yminmax,yminmax,100)
    
    U,V = np.meshgrid(u,v)
    Q = U+V*1j
    R=np.abs(p(Q)/q(Q))

    if fill:
        plt.pcolormesh(U, V, np.log(R),cmap=plt.get_cmap('seismic'),vmin=-4, vmax=4)
        plt.colorbar()
    if hatch:
        cs = plt.contourf(U,V,R,[0,1],hatches=[hatch],colors='none',alpha=1,extend='neither')
        cs.collections[0].set_ec(color)
        cs.collections[0].set_fc('none')

    plt.contour(U,V,R,[1],colors=[color],alpha=1,linewidths=linewith)
    plt.grid()
    
def calc_stab(rkm,bs):
    stab_functions = []
    for b in bs:
        display(b)
        rkm.b = b
        stab_functions.append(rkm.stability_function())
    return stab_functions

def calc_stab_exp(rkm,bs):
    stab_functions = []
    for b in bs:
        display(b)
        rkm.b = b
        stab_functions.append(rk.ExplicitRungeKuttaMethod(A=rkm.A,b=b).stability_function())
    return stab_functions

def color(chi,color1=[0,0,1],color2=[0,0,0.5]):
    return (chi*color1[0]+(1-chi)*color2[0],chi*color1[1]+(1-chi)*color2[1],chi*color1[2]+(1-chi)*color2[2])

# Implicit case

We are plotting a convex combination of original BE 3 extrapolation method and the embedded backward Euler scheme.

In [ ]:
ex3 = rk.extrap(3,'implicit euler').__num__()

chis = np.linspace(0,1,10)
bs = []
for chi in chis:
    bs.append(chi*ex3.b+(1-chi)*np.array([0,0,0,1/3,1/3,1/3]))


print('b[0]',bs[0])
print('b[-1]',bs[-1])    

stab_functions = calc_stab(ex3,bs)


In [ ]:
scale = 1.2
fig, ax = plt.subplots(1, 1, figsize=(scale*6.4, scale*4.8))

for i in range(1,9):
    plot_stab(stab_functions[i],(0.5,0.5,0.5))
    
plot_stab(stab_functions[0],'C0',hatch='/')
plot_stab(stab_functions[-1],'C1',hatch='\\')

be3 = mlines.Line2D([], [], color='C0', label='chain of BE')
ex3 = mlines.Line2D([], [], color='C1', label='BE 3 extrapolation')
altered = mlines.Line2D([], [], color=(0.5,0.5,0.5),label='convex combination')
legend_marker = plt.legend(handles=[be3,ex3,altered],loc='upper left')
plt.gca().add_artist(legend_marker)

plt.grid()
plt.savefig('stab_ex3.pdf', bbox_inches="tight")

# Explicit case

In [ ]:
dp5_or = rk.loadRKM('DP5').__num__()
dp5 = rk.ExplicitRungeKuttaMethod(A=dp5_or.A,b=dp5_or.b) #get rid of embedded method
b_org=dp5_or.b


In [ ]:
b = np.load('b_ex_adde.npy') #Data from the notebook Convergence.ipynb

In [ ]:
b.shape

In [ ]:
bs = []
norm = np.zeros((b.shape[2]))
for i in range(0,b.shape[2],1):
    b_i = b[:,0,i]
    bs.append(b_i)
    norm[i] =np.linalg.norm(b_i-b_org,ord=1)

In [ ]:
stab_functions = calc_stab_exp(dp5,bs)

In [ ]:
scale = 1.2
fig, ax = plt.subplots(1, 1, figsize=(scale*6.4, scale*4.8))

#setup colormap
cmap = plt.cm.viridis

i_max = np.argmax(norm)
display(i_max)

norm_color = lambda n: cmap(0.2+0.6*(n/norm[i_max]))

for i in range(len(bs)):
    print(i)
    display(stab_functions[i])
    plot_stab(stab_functions[i],norm_color(norm[i]),xmax=2,xmin=-6,yminmax=5,linewith=2)

dp5 = rk.ExplicitRungeKuttaMethod(A=dp5_or.A,b=dp5_or.b)
stab_orig = dp5.stability_function()
plot_stab(stab_orig,norm_color(0),xmax=2,xmin=-6,yminmax=5,hatch='/',linewith=3)
plt.grid(1)



stab_max=rk.ExplicitRungeKuttaMethod(A=dp5_or.A,b=bs[i_max]).stability_function()
plot_stab(stab_max,norm_color(norm[i_max]),xmax=2,xmin=-6,yminmax=5,hatch='\\',linewith=3)

#I = [[0.2,0.2+0.6]]
#im = plt.imshow(I, cmap='viridis',extent=(-1,0,-1,0),vmax=1,vmin=0)
#ax[0].colorbar(m, loc='ll', label='colorbar label')
#I = [[0.2,0.2+0.6],[0.2,0.2+0.6]]

#im = ax.pcolor([[1,0],[1,0]],[[0,0],[1,1]],I,cmap='viridis',extent=(-1,0,-1,0),vmax=1,vmin=0)

original = mlines.Line2D([], [], color=norm_color(0), label='Dormand--Prince RK5')
max_altered = mlines.Line2D([], [], color=norm_color(norm[i_max]), label=r'$\tilde{b} = $argmax$_{\tilde{b}} \| \tilde{b} - b \|_1$')
altered = mlines.Line2D([], [], color=(0.5,0.5,0.5),label=r'other $\tilde{b}$')
legend_marker = plt.legend(handles=[original,max_altered], loc='upper left')#ncol=2
#loc='center right'xmax=8,xmin=-5.5,yminmax=5.5
plt.gca().add_artist(legend_marker)


from mpl_toolkits.axes_grid1.inset_locator import inset_axes

axins1 = inset_axes(ax,
                    width="50%",  # width = 50% of parent_bbox width
                    height="5%",  # height : 5%
                    loc='lower left')


#im1 = ax.imshow([[1, 2], [2, 3]])
normalize = mpl.colors.Normalize(vmin=0, vmax=1)
cb1 = mpl.colorbar.ColorbarBase(axins1, cmap=cmap,
                                norm=normalize,
                                orientation='horizontal',
                               boundaries=np.linspace(0.2,0.8),
                               ticks=[0.2,0.8])
cb1.set_ticklabels(['0',str(round(norm[i_max],3))], update_ticks=True)
#fig.colorbar(im, cax=axins1, orientation="horizontal", ticks=[1, 2, 3])
axins1.xaxis.set_ticks_position("top")
axins1.xaxis.set_label("top")
#plt.text(60, .025, r'$ \| \tilde{b} - b \|_1$')
axins1.text(0.5,1.7,r'$ \| \tilde{b} - b \|_1$',horizontalalignment='center')

ax.set_xlim(-10,2)
ax.set_ylim(-5,6)

plt.savefig('stab_dp5.pdf', bbox_inches="tight")


In [ ]:
dp5 = rk.ExplicitRungeKuttaMethod(A=dp5_or.A,b=dp5_or.b)
stab_orig = dp5.stability_function()
plot_stab(stab_orig,(0,0,0.5,1),xmax=5,xmin=-5,yminmax=5)